In [47]:
%cd /content/drive/MyDrive/HASOC2021-EnglishHindiMarathi/trainingSet

/content/drive/MyDrive/HASOC2021-EnglishHindiMarathi/trainingSet


In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
pip install torch==1.3.1+cpu -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 111.8 MB 53 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.10.0+cu102 requires torch==1.9.0, but you have torch 1.3.1+cpu which is incompatible.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.3.1+cpu which is incompatible.


In [34]:
import pandas as pd
import numpy as np
from glob import glob
import re
import json
import tensorflow as tf
import torch
import pickle

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout,BatchNormalization,GRU

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import stemmer as hindi_stemmer

In [5]:
english_stopwords = stopwords.words("english")
with open('final_stopwords.txt', encoding = 'utf-8') as f:
    hindi_stopwords = f.readlines()
    for i in range(len(hindi_stopwords)):
        hindi_stopwords[i] = re.sub('\n','',hindi_stopwords[i])
stopwords = english_stopwords + hindi_stopwords
english_stemmer = SnowballStemmer("english")

In [6]:
df=pd.read_excel("hi_Hasoc2021_train.xlsx")

In [7]:
df.head()

,Unnamed: 0,_id,tweet_id,text,task_1,task_2
0,998,60c5d7495659ea5e55df0b7b,hi_hasoc_2021_998,@rssurjewala #Hindus DYING #HindusLivesMatter ...,NOT,NONE
1,4049,60c5d7495659ea5e55df1b73,hi_hasoc_2021_4049,सब लोग इतने पैसे डोनेट ही कर रहे हैं फिर भी आम...,NOT,NONE
2,1757,60c5d7495659ea5e55df0d1b,hi_hasoc_2021_1757,शेर-ए- सिवान शहाबुद्दीन साहब से रिश्ता क्या ل...,NOT,NONE
3,5175,60c5d7495659ea5e55df0e45,hi_hasoc_2021_5178,@AskAnshul आसमानी किताब के नाजायज औलाद है।,HOF,OFFN
4,1825,60c5d7495659ea5e55df0ee1,hi_hasoc_2021_1825,@Shikha0222 इसे कहते हैं दोगला पंती जिस सपा की...,NOT,NONE


In [8]:
tweets = df.text
y = df.task_1

In [9]:
regex_for_english_hindi_emojis="[^a-zA-Z#\U0001F300-\U0001F5FF'|'\U0001F600-\U0001F64F'|'\U0001F680-\U0001F6FF'|'\u2600-\u26FF\u2700-\u27BF\u0900-\u097F]"
def clean_tweet(tweet):
    tweet = re.sub(r"@[A-Za-z0-9]+",' ', tweet)
    tweet = re.sub(r"https?://[A-Za-z0-9./]+",' ', tweet)
    tweet = re.sub(regex_for_english_hindi_emojis,' ', tweet)
    tweet = re.sub("RT ", " ", tweet)
    tweet = re.sub("\n", " ", tweet)
    tweet = re.sub(r" +", " ", tweet)
    tokens = []
    for token in tweet.split():
        if token not in stopwords:
            token = english_stemmer.stem(token)
            tokens.append(token)
    return " ".join(tokens)

In [10]:
cleaned_tweets = [clean_tweet(tweet) for tweet in tweets]

In [11]:
cleaned_tweets

['#hindus die #hinduslivesmatt #kindattnpmnarendramodi #coronasecondwaveinindia #modimadedisast #resignmodi #bjp #rss liar i warn yrs back #rssterrorist ban by sardar patel ignor #शेर नहीं भेड़िया पाल भक्तो #mautkasaudagar',
 'सब लोग इतने पैसे डोनेट आम आदमी सिलेंडर कन्सेंट्रेटर ख़ुद ख़रीदना पड़ पैसे बीयर्ड ऑयल #indiafightscorona #indianeedoxygen #indiacovidcrisi',
 'शेर ए सिवान शहाबुद्दीन साहब रिश्ता #justiceforshahabuddin',
 'आसमानी किताब के नाजायज औलाद है।',
 'दोगला पंती सपा दम सफर माया इज्ज़त बचा पाई सपा मिटाने बात',
 'news बिहार आम जनता नाराजगी केंद्र सरकार सिस्टम लालू परिवार दिन काले अक्षरों लिखा जाएगा आने वाला पीढ़ी याद रखेगा संसद अपमानित मिट्टी नहीं मिलने हमारी बारी तुम्हारी बारी होगी #justiceforshahabuddin',
 'तेरी असलियत चोरी रोज करेगा खेत स्वीकार नहीं करेगा। गजेटियर तुझे दे तू कुत्ता कुत्ता रहा। भाग कुत्ते औलाद। कहीं भैंस गन्ने चोरी ले सुबह घर चलेगा तेरा।',
 'भोजपुरी टके सिंगर के लड़की नाम फूहड़ गाना बना सोचा वो लड़की घर निकलती मोहल्ले पान दुकान dj नाम फूहड़ गाना बज कैसा महसू

## **Distil Bert**

In [13]:
! pip install -q torch
! pip install -q transformers
! pip install -q inltk

     |████████████████████████████████| 2.8 MB 5.3 MB/s 
     |████████████████████████████████| 895 kB 56.0 MB/s 
     |████████████████████████████████| 3.3 MB 39.4 MB/s 
     |████████████████████████████████| 52 kB 1.3 MB/s 
     |████████████████████████████████| 636 kB 63.9 MB/s 
     |████████████████████████████████| 233 kB 6.0 MB/s 
     |████████████████████████████████| 1.3 MB 40.7 MB/s 
     |████████████████████████████████| 78 kB 5.7 MB/s 
     |████████████████████████████████| 1.2 MB 28.3 MB/s 
     |████████████████████████████████| 142 kB 55.7 MB/s 
     |████████████████████████████████| 294 kB 48.4 MB/s 
     |████████████████████████████████| 831.4 MB 2.0 kB/s 


In [14]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForMaskedLM.from_pretrained("distilbert-base-uncased")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [15]:
X = cleaned_tweets
cleaned_tweets = []

In [16]:
tokenized_input = tokenizer(
        X[0],
        padding=True,
        truncation=False,
        return_tensors='pt'
    )

sample_output = model(**tokenized_input)

In [17]:
print(tokenized_input['input_ids'].cpu().detach().numpy().shape)
print(tokenized_input['attention_mask'].cpu().detach().numpy().shape)
print(sample_output.logits.cpu().detach().numpy().shape)

(1, 87)
(1, 87)
(1, 87, 30522)


In [18]:
tokenized_input = tokenizer(
        X,
        padding=True,
        truncation=False,
        return_tensors='pt'
    )

In [19]:
tokenized_input['input_ids'] = tokenized_input['input_ids'].cpu().detach().numpy()
tokenized_input['attention_mask'] = tokenized_input['attention_mask'].cpu().detach().numpy()

In [23]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3)

In [29]:
input = {}
input['input_ids'] = torch.from_numpy(tokenized_input['input_ids'][0].reshape(1, 173))
input['attention_mask'] = torch.from_numpy(tokenized_input['attention_mask'][0].reshape(1, 173))

model_output = model(**input)
model_output = model_output.logits.cpu().detach().numpy()

In [30]:
model_output.shape

(1, 173, 30522)

In [25]:
tokenized_input['input_ids'].shape

(4594, 173)

In [35]:
output = []

j = 1

for i in range(len(X)):
    
    input = {}
    input['input_ids'] = torch.from_numpy(tokenized_input['input_ids'][i].reshape(1, 173))
    input['attention_mask'] = torch.from_numpy(tokenized_input['attention_mask'][i].reshape(1, 173))

    model_output = model(**input)
    model_output = model_output.logits.cpu().detach().numpy()
    output.append(model_output)
    if ((i + 1) % 200) == 0:
        output = np.array(output)
        output = output.reshape(len(output), 5280306)
        output = pca.fit_transform(output)
        file_name = "embed_feat_hiDB/output" + str(j) + ".txt"
        with open(file_name, "wb") as fp:
            pickle.dump(output, fp)
        print(file_name + " done")
        output = []
        j += 1

output = np.array(output)
output = output.reshape(len(output), 5280306)
output = pca.fit_transform(output)        
file_name = "embed_feat_hiDB/output" + str(j) + ".txt"
with open(file_name, "wb") as fp:
    pickle.dump(output, fp)
print(file_name + " done")
output = []

embed_feat_hiDB/output1.txt done
embed_feat_hiDB/output2.txt done
embed_feat_hiDB/output3.txt done
embed_feat_hiDB/output4.txt done
embed_feat_hiDB/output5.txt done
embed_feat_hiDB/output6.txt done
embed_feat_hiDB/output7.txt done
embed_feat_hiDB/output8.txt done
embed_feat_hiDB/output9.txt done
embed_feat_hiDB/output10.txt done
embed_feat_hiDB/output11.txt done
embed_feat_hiDB/output12.txt done
embed_feat_hiDB/output13.txt done
embed_feat_hiDB/output14.txt done
embed_feat_hiDB/output15.txt done
embed_feat_hiDB/output16.txt done
embed_feat_hiDB/output17.txt done
embed_feat_hiDB/output18.txt done
embed_feat_hiDB/output19.txt done
embed_feat_hiDB/output20.txt done
embed_feat_hiDB/output21.txt done
embed_feat_hiDB/output22.txt done
embed_feat_hiDB/output23.txt done


## Loading the model from the disk

In [39]:
output = []

for i in range(23):
    file_name = "./embed_feat_hiDB/output" + str(i + 1) + ".txt"
    with open(file_name, "rb") as fp:   #Pickling
        file_output = pickle.load(fp)
        for x in file_output:
            output.append(x)
    print(file_name + " done")

./embed_feat_hiDB/output1.txt done
./embed_feat_hiDB/output2.txt done
./embed_feat_hiDB/output3.txt done
./embed_feat_hiDB/output4.txt done
./embed_feat_hiDB/output5.txt done
./embed_feat_hiDB/output6.txt done
./embed_feat_hiDB/output7.txt done
./embed_feat_hiDB/output8.txt done
./embed_feat_hiDB/output9.txt done
./embed_feat_hiDB/output10.txt done
./embed_feat_hiDB/output11.txt done
./embed_feat_hiDB/output12.txt done
./embed_feat_hiDB/output13.txt done
./embed_feat_hiDB/output14.txt done
./embed_feat_hiDB/output15.txt done
./embed_feat_hiDB/output16.txt done
./embed_feat_hiDB/output17.txt done
./embed_feat_hiDB/output18.txt done
./embed_feat_hiDB/output19.txt done
./embed_feat_hiDB/output20.txt done
./embed_feat_hiDB/output21.txt done
./embed_feat_hiDB/output22.txt done
./embed_feat_hiDB/output23.txt done


In [40]:
X = output
output = []

In [41]:
print(len(X))

4594


In [48]:
with open("./embed_labels_hiDB/label.txt", "wb") as fp:
    pickle.dump(y, fp)

In [49]:
with open('./embed_labels_hi/label.txt', "rb") as fp:
        y = pickle.load(fp)

In [50]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

## **Logistic Regression**

In [53]:
# Preprocessing

X_train = np.array(X_train)
X_train = X_train.reshape(3675, 3)

X_val = np.array(X_val)
X_val = X_val.reshape(919, 3)

y_train = np.array(y_train)
y_val = np.array(y_val)

In [54]:
# Loading the Model

classifier = LogisticRegression()

In [55]:
classifier.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [56]:
# Evaluating the Model

y_pred = classifier.predict(X_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

         HOF       0.00      0.00      0.00       297
         NOT       0.68      1.00      0.81       622

    accuracy                           0.68       919
   macro avg       0.34      0.50      0.40       919
weighted avg       0.46      0.68      0.55       919



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## **Stochastic Gradient Descent**

In [57]:
from sklearn.linear_model import SGDClassifier

In [58]:
classifier = SGDClassifier()
classifier.fit(X_train, y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [59]:
y_pred = classifier.predict(X_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

         HOF       0.34      0.62      0.44       297
         NOT       0.70      0.44      0.54       622

    accuracy                           0.50       919
   macro avg       0.52      0.53      0.49       919
weighted avg       0.59      0.50      0.51       919



## **Support Vector Machine**

In [60]:
from sklearn.svm import SVC

In [61]:
classifier = SVC()
classifier.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [62]:
y_pred = classifier.predict(X_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

         HOF       1.00      0.00      0.01       297
         NOT       0.68      1.00      0.81       622

    accuracy                           0.68       919
   macro avg       0.84      0.50      0.41       919
weighted avg       0.78      0.68      0.55       919



## **Random Forest**

In [63]:
from sklearn.ensemble import RandomForestClassifier

In [64]:
rf_classi = RandomForestClassifier()
rf_classi.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [65]:
y_pred = rf_classi.predict(X_val)
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

         HOF       0.40      0.17      0.24       297
         NOT       0.69      0.88      0.77       622

    accuracy                           0.65       919
   macro avg       0.54      0.52      0.51       919
weighted avg       0.60      0.65      0.60       919



## **Ensembling**

In [66]:
# Importing Libraries

from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [67]:
# Logistic Regression

lr = LogisticRegression()
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_val)


# Support Vector Machine

svc = SVC()
svc.fit(X_train, y_train)
svc_pred = svc.predict(X_val)

# Naive Bayes

nb = GaussianNB()
nb.fit(X_train, y_train)
nb_pred = nb.predict(X_val)

# Stochastic Gradient Descent

sgd = SGDClassifier()
sgd.fit(X_train, y_train)
sgd_pred = sgd.predict(X_val)

# K Nearest Neighbour

knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_val)

# Decision Tree

dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
dt_pred = dt.predict(X_val)

# Random Forest

rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_val)

In [69]:
# Voting

y_pred = []

for i in range(len(lr_pred)):
    one = 0
    zero = 0
    #predictions = [lr_pred[i], svc_pred[i], sgd_pred[i], dt_pred[i], rf_pred[i]]
    predictions = [lr_pred[i], svc_pred[i], nb_pred[i], sgd_pred[i], knn_pred[i], dt_pred[i], rf_pred[i]]
    # predictions = [lr_pred[i], nb_pred[i], sgd_pred[i], dt_pred[i], rf_pred[i]]
    for pred in predictions:
        if pred == 'HOF': one += 1
        if pred == 'NOT': zero +=1
    if one > zero: y_pred.append('HOF')
    else: y_pred.append('NOT')

y_pred = np.array(y_pred)

In [70]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

         HOF       0.53      0.03      0.06       297
         NOT       0.68      0.99      0.81       622

    accuracy                           0.68       919
   macro avg       0.60      0.51      0.43       919
weighted avg       0.63      0.68      0.57       919

